# Chapter 8: Stochastic Methods

In [1]:
from copy import copy
import jax
import numpy as np
from scipy import stats

## Algorithm 8.1

In [18]:
class NoisyDescent:

    def __init__(self, submethod, sigma):
        self.submethod = submethod
        self.sigma = sigma
        self.k = 1
    
    def step(self, f, gradient, x):
        x = self.submethod.step(f, gradient, x)
        sigma = self.sigma[self.k]
        x += sigma*np.random.randn(x.shape[0])
        self.k += 1

        return x

### Example

In [19]:
class GradientDescent:
    def __init__(self, alpha):
        self.alpha = alpha # α

    def step(self, f, gradient, x):
        g = gradient(x)
        return x - self.alpha*g

def fun(x):
    return jax.numpy.sin(x[0]*x[1])+jax.numpy.exp(x[1]+x[2])-x[2]

x_0 = np.array([1.0, 2.0, 3.0])
gradient = jax.grad(fun)
submethod = GradientDescent(0.1)
sigma = [1.1/k for k in range(1,21)]
M = NoisyDescent(submethod, sigma)
x_1 = M.step(fun, gradient, x_0) 
print(x_1)

[  1.3609072 -12.642112  -12.549556 ]


## Algorithm 8.2

In [34]:
def rand_positive_spanning_set(alpha, n):
    delta = np.round(1/np.sqrt(alpha))
    L = np.diag(delta*np.random.choice([-1,1],size=n ))
    for i in range(0,n-1):
        for j in range(i+1,n):
            L[j,i] = np.random.uniform(-delta+1,delta-1)
    D = L[np.random.permutation(n),:]
    D = D[:, np.random.permutation(n)]
    D = np.concatenate([D, -np.sum(D,axis=1).reshape(-1,1)], axis=1)
    return [D[:,i] for i in range(0, n+1)]

## Algorithm 8.3

In [35]:
def mesh_adaptive_direct_search(f, x, epsilon):
    alpha, y, n = 1, f(x), x.shape[0]
    while alpha > epsilon:
        improved = False
        for i, d in enumerate(rand_positive_spanning_set(alpha, n)):
            x_prime = x + alpha*d
            y_prime = f(x_prime)
            if y_prime < y:
                x, y, improved = x_prime, y_prime, True
                x_prime = x + 3*alpha*d
                y_prime = f(x_prime)
                if y_prime < y:
                    x, y = x_prime, y_prime
        alpha = np.min([4*alpha,1]) if improved else alpha/4
    return x

### Example

In [38]:
def fun(x):
    return -np.exp(-(x[0]*x[1] - 1.5)**2 -(x[1]-1.5)**2)
x_0 = np.array([2.75, 2.25])
epsilon = 0.01
x_sol = mesh_adaptive_direct_search(fun, x_0, epsilon)
print(x_sol)

[0.98191814 1.61222439]


## Algorithm 8.4

In [2]:
def simulated_annealing(f, x, T, t, k_max):
    y = f(x)
    x_best, y_best = x, y
    for k in range(k_max):
        x_prime = x + T.rvs()
        y_prime = f(x_prime)
        delta_y = y_prime - y
        if delta_y <= 0 or np.random.random() < np.exp(-delta_y/t(k)):
            x, y = x_prime, y_prime
        
        if y_prime < y_best:
            x_best, y_best = x_prime, y_prime
        
    
    return x_best

### Example

In [4]:
def fun(x):
    return -np.exp(-(x[0]*x[1] - 1.5)**2 -(x[1]-1.5)**2)
x_0 = np.array([2.75, 2.25])
k_max = 2000
t = lambda k : 500.0/(k+1)
T = stats.norm()
x_sol = simulated_annealing(fun, x_0, T, t, k_max)
print(x_sol)

[1.56344716 1.06344716]


## Algorithm 8.5

In [ ]:
def corona_update(v, a, c, ns):
    for i in v.shape[0]:
        ai, ci = a[i], c[i]
        if ai > 0.6*ns:
            v[i] *= (1 + ci*(ai/ns - 0.6)/0.4)
        elif ai < 0.4*ns:
            v[i] /= (1 + ci*(0.4-ai/ns)/0.4)

    return v